In [2]:
import os
import re
import sys
import pandas as pd
sys.path.insert(0, './_modules')
from db_functions import TheDB
#from logging_module import getlogger
from logging_module_v2 import StartLogger
from regex_module import FindEmail, FindNumbersBraket, FindLawsuit, FindCaseId

from sqlalchemy import create_engine
import pymysql

In [3]:
## bots BD connections
conn_dict = {'host':'localhost', 'database': 'brspcqadev',
             'user':'root', 'password':'quito2020'}

cqdb = TheDB(conn_dict)

In [4]:
help(TheDB)

Help on class TheDB in module db_functions:

class TheDB(builtins.object)
 |  TheDB(conn_dict)
 |  
 |  Methods defined here:
 |  
 |  Conn(self)
 |      ##############################################
 |  
 |  GetTable(self, tablename, option='full')
 |      Get the table and converts to a pandas dataframe.
 |      Input: i) tablename
 |             ii) option = 'full' (full table) or option = column_name 
 |      Output: i) df
 |  
 |  InputW2V(self, df, text_column, basename, return_table=False)
 |      Inputs text data to the w2v_table in metabot DB
 |      Input: i) df: the df input
 |          ii) text_column: the name of the column that contains the interest text
 |          iii) basename: a label that identifies the origin of the text being inputed
 |          iv) db_object: the DB object
 |      Output: nothing
 |  
 |  InsertDB(self, df, table_db, key_column='', return_table=False, append_table=True)
 |      #####################################################################

In [3]:
data_path = os.path.join(os.getcwd(), '_data_output')

In [4]:
os.listdir(data_path)

['tmail_data_clean_bigrams_cluster_3_2020-03-01.csv',
 'tmail_data_clean_bigrams_cluster_22_2020-03-01.csv',
 'tmail_data_clean_bigrams_cluster_12_2020-03-01.csv',
 'tmail_data_clean_bigrams_cluster_9_2020-04-01.csv',
 'mail_data_clean_bigrams.csv',
 'tmail_data_clean_bigrams_cluster_16_2020-03-01.csv',
 'tmail_data_clean_bigrams_cluster_4_2020-03-01.csv',
 'tmail_data_clean_bigrams_cluster_9_2020-03-10.csv',
 'tmail_data_clean_bigrams_cluster_18_2020-03-01.csv',
 'tmail_data_clean_bigrams_cluster_15_2020-04-05.csv',
 'tmail_data_clean_bigrams_cluster_9_2020-03-04.csv',
 'tmail_data_clean_bigrams_cluster_20_2020-03-01.csv',
 'tmail_data_clean_bigrams_cluster_9_2020-03-26.csv',
 'tmail_data_clean_bigrams_cluster_8_2020-03-04.csv',
 'MainTrix_BoletosAds.csv',
 'mail_data_clean_bigrams_optimalcluster.csv',
 'tmail_data_clean_bigrams_cluster_2_2020-03-01.csv',
 'mail_data_clean_bigrams_optimalcluster_2020-04-01.csv',
 'tmail_data_clean_bigrams_cluster_8_2020-03-01.csv',
 'oficios_google_cl

In [5]:
df = pd.read_csv(os.path.join(data_path, 'main_trix_clean.csv'), sep = '\t')
print(df.columns)
df = df.rename(columns = {'Case # (DLBCA and Google reference)':'case_id',\
                     'line':'line_id', 'Lawsuit Number':'external_ref_number', 'Deadline':'deadline'})
df = df.loc[:, ['line_id', 'case_id', 'external_ref_number', 'deadline']]
df['external_ref_number'] = df['external_ref_number'].apply(FindLawsuit)
df['case_id'] = df['case_id'].apply(FindCaseId)

Index(['line', 'Case # (DLBCA and Google reference)', 'UID',
       'Date of the Court Orders', 'Court', 'same court', 'Lawsuit Number',
       'CP?', 'Summary of the request/what to produce [internal team rec]',
       'STQ confirm scope of production', 'Deadline',
       'DLBCA's suggested answer', 'Previous Orders and Answers',
       'Google's proposed answer', 'LIS: \nInformatin about case',
       'LIS: \nPolicy and content check',
       'LIS: \nGoogle Inc answer sent to authority? which one?',
       'LIS:\nPriority', 'LIS:\nAssignee', 'Status',
       'Status - writ of mandamus', 'Law Firm'],
      dtype='object')


In [6]:
df

,line_id,case_id,external_ref_number,deadline
0,1,,0009460-19.2013.403.6181,2014-02-28 00:00:00
1,2,6-9080000028545,0002618-91.2011.403.6181,2013-11-28 00:00:00
2,3,9-5821000001559,0007067-24.2013.403.6181,2013-10-22 00:00:00
3,4,5-6191000002205,,2013-11-22 00:00:00
4,5,9-9709000000828,0076690-72.2011.805.0001,2013-10-21 00:00:00
...,...,...,...,...
7614,7740,7-9812000028504,5075998-22.2019.4.04.7000,2019-12-24 00:00:00
7615,7741,6-8938000029130,0330591-87.2019.8.05.0001,2019-12-30 00:00:00
7616,7742,0-5628000028599,505-23.2019.4.01.3601,2020-01-03 00:00:00
7617,7743,0-5628000028599,505-23.2019.4.01.3601,2020-12-24 00:00:00


In [7]:
## input some date, since those fields are manadatory
df['date_received'] = '2020-08-12'
df['date_received_cq'] = '2020-08-12'

In [8]:
df = df[df['line_id'].str.isdigit() == True]
df['line_id'] = df['line_id'].astype(int)

/home/dev/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
df

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq
0,1,,0009460-19.2013.403.6181,2014-02-28 00:00:00,2020-08-12,2020-08-12
1,2,6-9080000028545,0002618-91.2011.403.6181,2013-11-28 00:00:00,2020-08-12,2020-08-12
2,3,9-5821000001559,0007067-24.2013.403.6181,2013-10-22 00:00:00,2020-08-12,2020-08-12
3,4,5-6191000002205,,2013-11-22 00:00:00,2020-08-12,2020-08-12
4,5,9-9709000000828,0076690-72.2011.805.0001,2013-10-21 00:00:00,2020-08-12,2020-08-12
...,...,...,...,...,...,...
7614,7740,7-9812000028504,5075998-22.2019.4.04.7000,2019-12-24 00:00:00,2020-08-12,2020-08-12
7615,7741,6-8938000029130,0330591-87.2019.8.05.0001,2019-12-30 00:00:00,2020-08-12,2020-08-12
7616,7742,0-5628000028599,505-23.2019.4.01.3601,2020-01-03 00:00:00,2020-08-12,2020-08-12
7617,7743,0-5628000028599,505-23.2019.4.01.3601,2020-12-24 00:00:00,2020-08-12,2020-08-12


In [11]:
df['line_id'] = df['line_id'].astype(int) + df_db_trix['line_id'].max()

/home/dev/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
df = df.drop_duplicates(subset = ['line_id'])

In [13]:
df['date_received'] = pd.to_datetime(df['date_received'])
df['date_received_cq'] = pd.to_datetime(df['date_received_cq'])

/home/dev/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/dev/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [159]:
df_test = df[15:].drop('line_id', axis=1)
df_test

,case_id,external_ref_number,deadline,date_received,date_received_cq
15,0-9431000002876,0007583-66.2013.403.6109,2014-03-14 00:00:00,2020-08-12,2020-08-12
16,3-8721000000094,0048139-19.2012.8.19.0014,NaN,2020-08-12,2020-08-12
17,3-2949000003517,0002607-76.2014.403.6110,2014-05-12 00:00:00,2020-08-12,2020-08-12
18,8-5005000003432,5013723-77.2014.404.7108,2014-05-14 00:00:00,2020-08-12,2020-08-12
19,3-4031000002808,0109447-16.2013.8.26.0050,NaN,2020-08-12,2020-08-12
...,...,...,...,...,...
7614,7-9812000028504,5075998-22.2019.4.04.7000,2019-12-24 00:00:00,2020-08-12,2020-08-12
7615,6-8938000029130,0330591-87.2019.8.05.0001,2019-12-30 00:00:00,2020-08-12,2020-08-12
7616,0-5628000028599,505-23.2019.4.01.3601,2020-01-03 00:00:00,2020-08-12,2020-08-12
7617,0-5628000028599,505-23.2019.4.01.3601,2020-12-24 00:00:00,2020-08-12,2020-08-12


In [160]:
df_model = pd.read_csv('model_trix_db.csv', sep = '\t')

In [161]:
#df_model = df_db_trix.copy()

In [177]:
df_model.T

,0
line_id,1
case_id,NaN
external_ref_number,NaN
deadline,0001-01-01 00:00:00
date_received,0001-01-01 00:00:00
date_received_cq,2019-06-17 17:22:28
auth_last_name,NaN
info_case,NaN
case_country,NaN
email_response,NaN


In [164]:
columns_list = list(df_model.columns)

In [165]:
df_clean = pd.DataFrame(columns = columns_list)

In [166]:
df_test

,case_id,external_ref_number,deadline,date_received,date_received_cq
15,0-9431000002876,0007583-66.2013.403.6109,2014-03-14 00:00:00,2020-08-12,2020-08-12
16,3-8721000000094,0048139-19.2012.8.19.0014,NaN,2020-08-12,2020-08-12
17,3-2949000003517,0002607-76.2014.403.6110,2014-05-12 00:00:00,2020-08-12,2020-08-12
18,8-5005000003432,5013723-77.2014.404.7108,2014-05-14 00:00:00,2020-08-12,2020-08-12
19,3-4031000002808,0109447-16.2013.8.26.0050,NaN,2020-08-12,2020-08-12
...,...,...,...,...,...
7614,7-9812000028504,5075998-22.2019.4.04.7000,2019-12-24 00:00:00,2020-08-12,2020-08-12
7615,6-8938000029130,0330591-87.2019.8.05.0001,2019-12-30 00:00:00,2020-08-12,2020-08-12
7616,0-5628000028599,505-23.2019.4.01.3601,2020-01-03 00:00:00,2020-08-12,2020-08-12
7617,0-5628000028599,505-23.2019.4.01.3601,2020-12-24 00:00:00,2020-08-12,2020-08-12


In [167]:
df_final = pd.concat([df_clean, df_test], sort = False).loc[:, columns_list].fillna('')
df_final

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq,auth_last_name,info_case,case_country,email_response,...,date_status_ms,deadline_ms,ms_info,letter_reviewed,user_in_charge_production,production_completed,production_ready,monitor,emissary_chosen,closure
15,,0-9431000002876,0007583-66.2013.403.6109,2014-03-14 00:00:00,2020-08-12,2020-08-12,,,,,...,,,,,,,,,,
16,,3-8721000000094,0048139-19.2012.8.19.0014,,2020-08-12,2020-08-12,,,,,...,,,,,,,,,,
17,,3-2949000003517,0002607-76.2014.403.6110,2014-05-12 00:00:00,2020-08-12,2020-08-12,,,,,...,,,,,,,,,,
18,,8-5005000003432,5013723-77.2014.404.7108,2014-05-14 00:00:00,2020-08-12,2020-08-12,,,,,...,,,,,,,,,,
19,,3-4031000002808,0109447-16.2013.8.26.0050,,2020-08-12,2020-08-12,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7614,,7-9812000028504,5075998-22.2019.4.04.7000,2019-12-24 00:00:00,2020-08-12,2020-08-12,,,,,...,,,,,,,,,,
7615,,6-8938000029130,0330591-87.2019.8.05.0001,2019-12-30 00:00:00,2020-08-12,2020-08-12,,,,,...,,,,,,,,,,
7616,,0-5628000028599,505-23.2019.4.01.3601,2020-01-03 00:00:00,2020-08-12,2020-08-12,,,,,...,,,,,,,,,,
7617,,0-5628000028599,505-23.2019.4.01.3601,2020-12-24 00:00:00,2020-08-12,2020-08-12,,,,,...,,,,,,,,,,


In [168]:
df_final['status'] = df_model['status'][0]
df_final['label'] = df_model['label'][0]
df_final['date_created'] = df_model['date_created'][0]
df_final['date_modified'] = df_model['date_modified'][0]
df_final['users_id'] = df_model['users_id'][0]
df_final['user_advice_correction'] = df_model['user_advice_correction'][0]
df_final['advice_approved'] = df_model['advice_approved'][0]
df_final['advice_sent'] = df_model['advice_sent'][0]
df_final['petition_cat_id'] = df_model['petition_cat_id'][0]
df_final['date_status_ms'] = df_model['date_status_ms'][0]
df_final['deadline_ms'] = df_model['deadline_ms'][0]
df_final['user_in_charge_production'] = df_model['user_in_charge_production'][0]
df_final['monitor'] = df_model['monitor'][0]
df_final['emissary_chosen'] = df_model['emissary_chosen'][0]



## fillna
df_final['deadline'].fillna('2020-08-20', inplace = True)

In [169]:
df_final

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq,auth_last_name,info_case,case_country,email_response,...,date_status_ms,deadline_ms,ms_info,letter_reviewed,user_in_charge_production,production_completed,production_ready,monitor,emissary_chosen,closure
15,,0-9431000002876,0007583-66.2013.403.6109,2014-03-14 00:00:00,2020-08-12,2020-08-12,,,,,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
16,,3-8721000000094,0048139-19.2012.8.19.0014,,2020-08-12,2020-08-12,,,,,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
17,,3-2949000003517,0002607-76.2014.403.6110,2014-05-12 00:00:00,2020-08-12,2020-08-12,,,,,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
18,,8-5005000003432,5013723-77.2014.404.7108,2014-05-14 00:00:00,2020-08-12,2020-08-12,,,,,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
19,,3-4031000002808,0109447-16.2013.8.26.0050,,2020-08-12,2020-08-12,,,,,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7614,,7-9812000028504,5075998-22.2019.4.04.7000,2019-12-24 00:00:00,2020-08-12,2020-08-12,,,,,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
7615,,6-8938000029130,0330591-87.2019.8.05.0001,2019-12-30 00:00:00,2020-08-12,2020-08-12,,,,,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
7616,,0-5628000028599,505-23.2019.4.01.3601,2020-01-03 00:00:00,2020-08-12,2020-08-12,,,,,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
7617,,0-5628000028599,505-23.2019.4.01.3601,2020-12-24 00:00:00,2020-08-12,2020-08-12,,,,,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,


In [170]:
df_final['deadline'] = df_final['deadline'].replace('', '2020-08-20')
df_final['deadline'] = pd.to_datetime(df_final['deadline'])

In [171]:
df_final = df_final.drop('line_id', axis = 1)

In [172]:
df_final

,case_id,external_ref_number,deadline,date_received,date_received_cq,auth_last_name,info_case,case_country,email_response,status,...,date_status_ms,deadline_ms,ms_info,letter_reviewed,user_in_charge_production,production_completed,production_ready,monitor,emissary_chosen,closure
15,0-9431000002876,0007583-66.2013.403.6109,2014-03-14,2020-08-12,2020-08-12,,,,,Waiting Advice,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
16,3-8721000000094,0048139-19.2012.8.19.0014,2020-08-20,2020-08-12,2020-08-12,,,,,Waiting Advice,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
17,3-2949000003517,0002607-76.2014.403.6110,2014-05-12,2020-08-12,2020-08-12,,,,,Waiting Advice,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
18,8-5005000003432,5013723-77.2014.404.7108,2014-05-14,2020-08-12,2020-08-12,,,,,Waiting Advice,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
19,3-4031000002808,0109447-16.2013.8.26.0050,2020-08-20,2020-08-12,2020-08-12,,,,,Waiting Advice,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7614,7-9812000028504,5075998-22.2019.4.04.7000,2019-12-24,2020-08-12,2020-08-12,,,,,Waiting Advice,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
7615,6-8938000029130,0330591-87.2019.8.05.0001,2019-12-30,2020-08-12,2020-08-12,,,,,Waiting Advice,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
7616,0-5628000028599,505-23.2019.4.01.3601,2020-01-03,2020-08-12,2020-08-12,,,,,Waiting Advice,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
7617,0-5628000028599,505-23.2019.4.01.3601,2020-12-24,2020-08-12,2020-08-12,,,,,Waiting Advice,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,


In [176]:
df_final.columns

Index(['case_id', 'external_ref_number', 'deadline', 'date_received',
       'date_received_cq', 'auth_last_name', 'info_case', 'case_country',
       'email_response', 'status', 'targets', 'lers', 'label', 'date_created',
       'date_modified', 'users_id', 'legal_opinion_background',
       'legal_opinion_legal_risk', 'legal_opinion_additional_info',
       'legal_opinion_issue_comments', 'legal_opinion_silo',
       'legal_opinion_petition_comments', 'user_advice_correction',
       'council_review_letter', 'writ_filed_date', 'daily_fines',
       'freezing_assets_date', 'advice_approved', 'advice_pending_yes_no',
       'advice_pendency', 'advice_pendency_solved', 'advice_sent',
       'petition_cat_id', 'petition_yes_no_pending', 'petition_no_explanation',
       'petition_body', 'petition_draft_prepared', 'petition_draft_corrected',
       'petition_field', 'ms_draft_write', 'ms_draft_reviewed', 'ms_filed',
       'status_ms', 'date_status_ms', 'deadline_ms', 'ms_info',
       'l

In [174]:
## create sqlalchemy engine
engine = create_engine("mysql://{user}:{pw}@{host}/{db}?charset=utf8mb4"
                       .format(user= conn_dict['user'],
                               pw=conn_dict['password'],
                               host = conn_dict['host'],
                               db=conn_dict['database']), encoding="utf8")

In [175]:
df_final.to_sql('Trix', con = engine, if_exists = 'append',\
            index = False, chunksize = 1000)